<a href="https://colab.research.google.com/github/BonnieAo/ECON211-Week2-Decision-Tree/blob/main/Income_DecisionTree_ZiqiaoAo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1: Data Wrangling

In [ ]:
import pandas as pd 
import numpy as np
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn import metrics 
clf = DecisionTreeClassifier()

In [ ]:
###show all columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
df=pd.read_csv('https://raw.githubusercontent.com/BonnieAo/ECON211-Week2-Decision-Tree/main/adult.csv')
df.head()

In [ ]:
###deal with missing values and unresonable values
df = df[(df != '?').all(axis=1)]
df.head()

In [ ]:
df['income']=df['income'].apply(lambda x: '<=50K' if x== '<=50K' else '>50K')

In [ ]:
df['sex_num']=df['sex'].apply(lambda x: 1 if x== 'Female' else 2)

In [ ]:
df['race_num']=df['race'].apply(lambda x: 1 if x== 'White' else 2 if x== 'Other' else 3 if x== 'Asian-Pac-Islander' else 4 if x== 'Amer-Indian-Eskimo' else 5)

In [ ]:
df.head()

# Build Decision Tree

In [ ]:
feature_names=['race_num','sex_num','education.num']
#feature_names=['race_num','sex_num','education.num','age','capital.gain','capital.loss']
features=df[feature_names]
targets=df['income']
targets_names = targets.unique()

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV, cross_val_score,KFold

In [ ]:
### Find the max_depth using cross validation score
ScoreAll = []
cv = StratifiedShuffleSplit(n_splits=10, test_size=0.1, random_state=42)
for i in range(1, 20):
    clf = DecisionTreeClassifier(criterion='entropy', max_depth=i, random_state=42)
    score = cross_val_score(clf, features, targets, cv=cv).mean()
    ScoreAll.append([i, score])
ScoreAll = np.array(ScoreAll)
max_score = np.where(ScoreAll==np.max(ScoreAll[:, 1]))[0][0]
optimal_max_depth = ScoreAll[np.where(ScoreAll==np.max(ScoreAll[:, 1]))[0][0]][0]
print("optimal max_depth and max score:", ScoreAll[max_score])
import matplotlib.pyplot as plt
plt.figure(figsize=[20, 5])
plt.plot(ScoreAll[:, 0], ScoreAll[:, 1])
plt.show()

In [ ]:
###training & test data split
from sklearn.model_selection import train_test_split
train_features, test_features, train_targets, test_targets = train_test_split(features, targets, test_size=0.2, random_state=42)

In [ ]:
from sklearn import metrics
from sklearn.metrics import accuracy_score

In [ ]:
#####Optimizing Decision Tree Performance
# Create Decision Tree classifer object
clf = DecisionTreeClassifier(criterion='entropy', max_depth = optimal_max_depth,random_state=42)

# Train Decision Tree Classifer
clf = clf.fit(train_features,train_targets)

#Predict the response for test dataset
y_pred = clf.predict(test_features)

# Compute test set accuracy  
acc = accuracy_score(y_pred, test_targets)
print("Test set accuracy: {:.4f}".format(acc))

In [ ]:
from sklearn.tree import export_graphviz
from sklearn.externals.six import StringIO  
from IPython.display import Image  
import pydotplus

In [ ]:
### Plot the figure of decision tree
dot_data = StringIO()

export_graphviz(clf, out_file=dot_data,  

                filled=True, rounded=True,

                special_characters=True,feature_names = feature_names,class_names=['lower_than_50K','higher_than_50K'])

graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  

graph.write_png('ESG.png')

Image(graph.create_png())

# Build Random Forest


In [ ]:
from sklearn.ensemble import RandomForestClassifier
Forest = RandomForestClassifier(n_estimators = 25)
Forest = Forest.fit(train_features,train_targets)
y_pred = Forest.predict(test_features)
acc = accuracy_score(y_pred, test_targets)
print("Test set accuracy: {:.4f}".format(acc))
